
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# TensorFlow Lite Conversion #

<img src="https://miro.medium.com/max/1400/1*AO4WEkrSRz4Xb7qzkbqcvg.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

### Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


### Install required libraries

In [2]:
!pip install -U --pre tensorflow-gpu=="1.*"
!pip install tf_slim
!nvidia-smi
!pip install numpy==1.16 #This version is required due to a bug. After installing this version, the runtime has to be restarted.

     |████████████████████████████████| 411.0MB 37kB/s 
     |████████████████████████████████| 3.8MB 37.7MB/s 
     |████████████████████████████████| 512kB 44.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=d3da5e284da2bff030d4e7b2df5d78c63b18895a1d60eaa9f608ad94df42cdea
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling te

In [1]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import *

--2020-06-13 14:28:05--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5187 (5.1K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   5.07K  --.-KB/s    in 0s      

2020-06-13 14:28:05 (32.8 MB/s) - ‘constants.py’ saved [5187/5187]



In [0]:
!pip install pycocotools

!apt-get update && apt-get upgrade -y
!apt-get install python-pil python-lxml 

!pip install Cython
!pip install contextlib2
!pip install jupyter
!pip install matplotlib
!pip install wget

import os
import pathlib
import wget


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubu

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .

In [0]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import *

os.environ['PYTHONPATH'] += ":/content/models/research/:/content/models/research/slim"

### Prepare dataset ###

#### Copy dataset from Google Drive and extract

In [0]:
!gsutil cp {DRIVE_DEV}model_graph.zip {OBJ_DET_DIR}
!unzip -j {OBJ_DET_DIR}/model_graph.zip -d {INF_GRAPH}

!gsutil cp {DRIVE_DEV}model_checkpoints.zip {OBJ_DET_DIR}
!unzip -j {OBJ_DET_DIR}/model_checkpoints.zip -d {CKPT_DIR}

Copying file:///content/drive/My Drive/Machine Learning/Projects/Face-Mask-Detection/Dev/model_graph.zip...
-
Operation completed over 1 objects/50.6 MiB.                                     
Archive:  /content/tensorflow/workspace/model_graph.zip
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/checkpoint  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/pipeline.config  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/model.ckpt.meta  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/model.ckpt.data-00000-of-00001  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph/saved_model.pb  
  inflating: /content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/inference_graph

### Export inference graph

Use this option if you want to convert to TFjs later

In [0]:
!python {BASE_DIR}/models/research/object_detection/export_inference_graph.py \
  --pipeline_config_path="{CONFIG_DIR}/pipeline.config" \
  --trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-18560" \
  --output_directory="{INF_GRAPH}" \
  --add_postprocessing_op=true \
  --input_type=image_tensor \
  

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0611 10:44:29.284205 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/export_inference_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0611 10:44:29.293980 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/exporter.py:348: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0611 10:44:29.294343 139872719275904 module_wrapper.py:139] From /content/tensorflow/models/research/object_detection/exporter.py:113: The name tf.placeholder is deprecated. Please 

### Export TF Lite SSD inference graph

Use this option for TF Lite conversion

In [0]:
!python {BASE_DIR}/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="{CONFIG_DIR}/pipeline.config" \
--trained_checkpoint_prefix="{CKPT_DIR}/model.ckpt-19611" \
--output_directory="{TF_LITE_GRAPH}" \
--add_postprocessing_op=true

2020-06-11 21:21:50.333863: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-11 21:21:54.593521: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying strip_unused_nodes


### Convert graph to TF Lite

In [0]:
!tflite_convert \
--output_file="{TF_LITE_MODEL}/detect2.tflite" \
--graph_def_file="{TF_LITE_GRAPH}/tflite_graph.pb" \
--inference_type=QUANTIZED_UINT8 \
--input_arrays="{INPUT_TENSORS}" \
--output_arrays="{OUTPUT_TENSORS}"  \
--mean_values=128 \
--std_dev_values=128 \
--input_shapes=1,300,300,3 \
--change_concat_input_ranges=false \
--allow_nudging_weights_to_use_fast_gemm_kernel=true \
--allow_custom_ops \
--output_format=TFLITE \
--default_ranges_min=0 \
--default_ranges_max=255

2020-06-11 21:22:11.079323: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


## Save 

#### Archive the inference graph

In [0]:
!zip -r inf-graphv1.zip {INF_GRAPH}
!gsutil cp inf-graphv1.zip {DRIVE_DEV}

/content/training_demo
--2020-06-05 08:59:06--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4766 (4.7K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.65K  --.-KB/s    in 0s      

2020-06-05 08:59:06 (46.5 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4766/4766]



#### Archive the TF Lite graph

In [0]:
!zip -r tflite-graphv1.zip {TF_LITE_GRAPH}
!gsutil cp tflite-graphv1.zip {DRIVE_DEV}

  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/ (stored 0%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/tflite_graph.pb (deflated 8%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained_graphs/tf_lite_graph/tflite_graph.pbtxt (deflated 56%)
Copying file://tflite-graphv1.zip...
-
Operation completed over 1 objects/39.5 MiB.                                     


#### Archive the TF Lite model

In [0]:
!zip -r tflite-modelv4.zip {TF_LITE_MODEL}
!gsutil cp tflite-modelv4.zip {DRIVE_DEV}

  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained-models/tflite/ (stored 0%)
  adding: content/tensorflow/workspace/Face-Mask-Detection/training/model/trained-models/tflite/detect2.tflite (deflated 39%)
Copying file://tflite-modelv4.zip...
/ [1 files][  2.8 MiB/  2.8 MiB]                                                
Operation completed over 1 objects/2.8 MiB.                                      
